<a href="https://colab.research.google.com/github/Asma-Ahmed-Aqil-AL-Zubaidi/week_6_exam/blob/main/My_YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 16.2 MB/s eta 0:00:00


In [3]:
import cv2
import torch
from google.colab.patches import cv2_imshow

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [4]:
from ultralytics import YOLO
model = YOLO(model="/content/drive/MyDrive/T5/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt")

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [5]:
Vid_path='/content/drive/MyDrive/T5/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Video.mp4'
cap=cv2.VideoCapture(Vid_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [6]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

In [7]:
print(f"Video Height: {height}, Video Width: {width}, FPS: {fps}")

Video Height: 1080, Video Width: 1920, FPS: 29.97002997002997


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [8]:
output_path = 'output_with_detections.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_path,fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [9]:
left_count = 0
right_count = 0
middle_line_position = width // 2

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [10]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    results = model(frame)
    for result in results:
        for box in result.boxes:

            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            conf = box.conf.cpu().numpy()[0]
            cls = int(box.cls.cpu().numpy()[0])

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)


            label = f"ID: {cls}, Conf: {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


            if (x1 + x2) // 2 < middle_line_position:
                left_count += 1
            else:
                right_count += 1


    cv2.line(frame, (middle_line_position, 0), (middle_line_position, height), (0, 0, 255), 2)


    cv2.putText(frame, f"Left side vehicles : {left_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f"Right side vehicles: {right_count}", (width - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, "Asmaa alzobidi", (width - 200, 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


    output_video.write(frame)



cap.release()
output_video.release()




0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 1050.5ms
Speed: 23.4ms preprocess, 1050.5ms inference, 34.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 829.4ms
Speed: 8.2ms preprocess, 829.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 833.4ms
Speed: 3.6ms preprocess, 833.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 845.3ms
Speed: 4.1ms preprocess, 845.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 937.5ms
Speed: 13.0ms preprocess, 937.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 980.6ms
Speed: 4.1ms preprocess, 980.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 966.6ms
Speed: 3.2ms preproces

## Save and Submit Your Work

In [11]:
from google.colab import files

In [13]:
files.download('output_with_detections.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit both the notebook and the output video

steps

1- In the beginning, I did this programming thing(!pip install ultralytics) ،This allows me to use a pre-trained Moodle, and use(YOLO)

2-I have linked Google Colab with Google Drive

3-The goal of this step is to use the content in Google Drive and also upload files to Google Colab, such as the results and training date. It depends on the current project in our project. We will upload the video clip after specifying the objects in it.

4- The fourth step is to use the pre-trained model attached to the test file،Usually, this step was performed on other models,The purpose of this step is to reduce GPU consumption and speed up the task of writing code. Pre-trained models in general always give higher accuracy and consume less GPU.

5-The fifth step focuses on taking information about the video from which objects are to be detected, including length, width, and frame per second
the ruslt Video Height: 1080, Video Width: 1920, FPS: 29.97002997002997

6-The sixth step is the most important step in extracting the result. We determine the extension of the clip that will be the result after selecting the objects.Specify the extension for the resulting video, in our current example it is MP4

7-Displaying boxes for the objects, I wrote my name at the top, and wrote an identification number for each object،Run the model on the video clip, specifying a command to continue as long as the video clip is still playing

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
 This determines the operation of the mediator as long as the video clip is running


 8-The last step is to save the clip after specifying the object on it, and output it as a new video. The new video explains or determines the number of cars on the road and then tracks them, and this is an important point. It not only determines the location, but also tracks it until it exits the frame into the clip.





*   in the future, I will make some more modifications. I feel that I will not
set an identifier for each car if its location differs in the time frame. Rather, I will develop my code so that it identifies one car and assigns it the same number. In fact, this is another advantage. I can make this code more closely related to the location of the car, since Each car receives an identification number for a specific period of time.
*  Also, in improving this code, I will make it classify the type of traffic, large and small, to determine the permissible speed, not using artificial intelligence.


*   It is also possible to allow the code to take the street name from Google and put it in the frames of the quicker to facilitate data classification and analysis.







In this link you will find the code before  Object Detection that are here, the cars on the street, and after discovering the objects. Thank you
### :)

https://drive.google.com/drive/folders/1BDay5S7jl52WKCbP7LEQUxAv_FQfEDLI?usp=sharing